# Tarea 2 — Inteligencia Artificial (UDP)

**Instrucciones clave (resumen de la pauta):**  
- **Clustering**: K-Means, K-Means++ y MeanShift, **al menos 4 configs por técnica**, entrenar con **80%** (sin Y), evaluar con métrica (p.ej., Silhouette) y **elegir top-3**; aplicar al **20%**, asignar etiqueta por **clase dominante del cluster** y discutir razonabilidad.  
- **Supervisado**: Entrenar en **paralelo** múltiples instancias (≥3 por técnica) de **Regresión Logística** y **SVM**, **variando hiperparámetros** definidos en un **archivo de configuración externo**; entrenar con **80%** y **cada 5 épocas eliminar la peor**; para las **mejores 2 configs**, reportar métricas en **20%**.  

> **Nota académica:** Esta notebook contiene **código y estructura**. Debes **redactar tu propio análisis** en las celdas marcadas como *[Completar análisis]*. 


## 0) Setup del entorno (local)

1. Asegúrate de tener un entorno de Python 3.10+ (puede ser `conda` o `venv`).  
2. Instala dependencias en tu máquina:
```bash
pip install -r ../requirements.txt
```
Si usas VS Code, puedes abrir esta carpeta y ejecutar esta notebook con la extensión de Jupyter.


In [ ]:
import sys
print(sys.executable)              # Debe mostrar /usr/bin/python
import sklearn, numpy, scipy
print("sklearn:", sklearn.__version__)
print("numpy:", numpy.__version__, "scipy:", scipy.__version__)


In [4]:
# Imports principales
import os, json, yaml
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.metrics import silhouette_score

from src.utils import read_config, load_dataset, build_preprocessor, split_xy, make_splits
from src.clustering import run_clustering
from src.supervised import train_with_elimination

np.random.seed(42)


ModuleNotFoundError: No module named 'sklearn'

## 1) Cargar configuración y dataset

- Edita `../config/config.yaml` para ajustar:
  - `dataset_path`: ruta a tu CSV (por ejemplo, `data/ai4i_2020.csv` descargado de Kaggle).
  - `target_column`: nombre exacto de la columna Y (p.ej., `Failure Type`).
  - `drop_columns`, `categorical_columns`: ajusta de acuerdo a tu archivo.


In [ ]:
CFG_PATH = str(Path('../config/config.yaml'))
cfg = read_config(CFG_PATH)
cfg


In [ ]:
# Cargar CSV y chequear columnas
df, target_col, num_cols, cat_cols = load_dataset(cfg)
print(f"Shape: {df.shape}")
print("Columnas:", df.columns.tolist())
print("Objetivo (Y):", target_col)
print("Categóricas detectadas:", cat_cols)
print("Numéricas:", num_cols)

# Resumen de clases (para verificar multiclase)
print("\nDistribución de la etiqueta Y:")
print(df[target_col].value_counts(dropna=False))


## 2) Separar train/test (80/20) y preparar *preprocessor*


In [ ]:
X, y = split_xy(df, target_col)
X_train, X_test, y_train, y_test = make_splits(X, y, test_size=cfg['test_size'], random_state=cfg['random_state'])
print(X_train.shape, X_test.shape)

# El preprocesador se ajusta solo con train (para evitar leakage)
pre = build_preprocessor(num_cols, cat_cols)
Xtr = pre.fit_transform(X_train)
Xte = pre.transform(X_test)
Xtr.shape, Xte.shape


## 3) Clustering: K-Means / K-Means++ / MeanShift

- Entrenamos **solo con X_train** (sin Y), probando ≥4 configuraciones por técnica.
- Evaluamos con la métrica elegida (p.ej. *Silhouette*). Elegimos el **Top-3** y medimos la **asignación por etiqueta dominante** en el 20% de test.


In [ ]:
df_results, top3, pre_used = run_clustering(cfg, X_train, y_train, X_test, y_test, num_cols, cat_cols)
print("== Resultados (ordenado por métrica en train) ==")
display(df_results.head(12))
print("\n== TOP-3 por métrica en train ==")
display(top3)


### [Completar análisis] — *Clustering*
- Comenta brevemente las configuraciones que obtuvieron mejor puntaje (métrica en train).
- Discute si la asignación de **etiqueta dominante por cluster** al 20% de test parece razonable para imputar etiquetas faltantes.
- Agrega gráficos si lo estimas necesario (p.ej., distribución de tamaños de cluster). **No uses IA generativa para redactar este análisis.**


## 4) Modelos Supervisados (Logistic / SVM) con eliminación periódica

- Entrenamos **en paralelo** múltiples configuraciones (definidas en `config.yaml`), **solo con train**.
- Cada **5 épocas**, eliminamos la **peor** configuración (según métrica en *train*).
- Reportamos métricas en **test** para las **dos mejores** configuraciones finales.


In [ ]:
# (Opcional) Puedes ajustar n_jobs según tu CPU; -1 usa todos los núcleos
train_history, test_summary, survivors = train_with_elimination(cfg, Xtr, y_train, Xte, y_test, class_names=None, n_jobs=-1)

print("== Historial de evaluación (train) ==")
display(train_history)

print("\n== Métricas en test para las 2 mejores configuraciones ==")
display(test_summary[["name","algo","test_accuracy","test_f1_macro"]])

# Guardar resultados
Path("outputs").mkdir(exist_ok=True)
train_history.to_csv("outputs/train_history.csv", index=False)
test_summary.to_csv("outputs/test_summary.csv", index=False)
print("Archivos guardados en outputs/: train_history.csv y test_summary.csv")


### [Completar análisis] — *Supervisado*
- Explica cómo afectaron los **hiperparámetros** al rendimiento (por ejemplo, `alpha`, `learning_rate`, `loss`).
- Compara **Regresión Logística** vs **SVM** en las métricas de test (accuracy, F1 macro).
- Comenta brevemente el efecto de la **eliminación cada 5 épocas** sobre la convergencia de las configuraciones.
